# Simon’s Algorithm 

## 1. Problem Definition

We are given oracle (black-box) access to a function  
$f : \{0,1\}^n \rightarrow \{0,1\}^n$  
with the promise that **exactly one** of the following cases holds:

- **One-to-One (Injective)**  
  Every input maps to a unique output.  
  This corresponds to the trivial secret string  
  $s = 00\cdots0$.

- **Two-to-One (Simon’s Promise)**  
  For every output value, there are exactly two distinct inputs  
  $x_1, x_2 \in \{0,1\}^n$ such that  
  $f(x_1) = f(x_2)$,  
  and these inputs are related by a fixed but unknown **secret string** $s \neq 0$:
  $$
  x_2 = x_1 \oplus s \quad \text{or equivalently} \quad x_1 \oplus x_2 = s.
  $$

**Goal:**  
- Determine whether $f$ is one-to-one, **or**
- If $f$ is two-to-one, determine the secret string $s$.

---

## 2. Quantum Circuit Description

Simon’s algorithm uses **two registers**, each consisting of $n$ qubits:
- First register: input register
- Second register: output register

---

### Step 1: Initialization

Both registers are initialized to the all-zero state:
$$
|\psi_0\rangle = |0\rangle^{\otimes n} |0\rangle^{\otimes n}.
$$

---

### Step 2: Creating Superposition

Apply a Hadamard gate to each qubit of the **first register**:
$$
H^{\otimes n} |0\rangle^{\otimes n}
= \frac{1}{\sqrt{2^n}} \sum_{x \in \{0,1\}^n} |x\rangle.
$$

The joint state becomes:
$$
|\psi_1\rangle
= \frac{1}{\sqrt{2^n}} \sum_{x \in \{0,1\}^n} |x\rangle |0\rangle^{\otimes n}.
$$

This represents a uniform superposition over all possible inputs.

---

### Step 3: Oracle Transformation $U_f$

The oracle implements the reversible transformation:
$$
U_f : |x\rangle |y\rangle \mapsto |x\rangle |y \oplus f(x)\rangle.
$$

Since the second register is initialized to $|0\rangle$, this simplifies to:
$$
|x\rangle |0\rangle \mapsto |x\rangle |f(x)\rangle.
$$

Applying $U_f$ to $|\psi_1\rangle$ gives:
$$
|\psi_2\rangle
= \frac{1}{\sqrt{2^n}} \sum_{x \in \{0,1\}^n} |x\rangle |f(x)\rangle.
$$

Note that there is **no phase kickback** in Simon’s algorithm; all information is stored in correlations between registers.

---

### Step 4: Final Hadamard Transform

Apply $H^{\otimes n}$ again to the **first register** only.

Using the identity:
$$
H^{\otimes n} |x\rangle
= \frac{1}{\sqrt{2^n}} \sum_{z \in \{0,1\}^n} (-1)^{x \cdot z} |z\rangle,
$$
where $x \cdot z = x_1 z_1 \oplus x_2 z_2 \oplus \cdots \oplus x_n z_n$,

we obtain:
$$
|\psi_3\rangle
= \frac{1}{2^n}
\sum_{x \in \{0,1\}^n}
\sum_{z \in \{0,1\}^n}
(-1)^{x \cdot z}
|z\rangle |f(x)\rangle.
$$

---

## 3. Measurement Analysis

### Case A: One-to-One Function ($s = 0$)

Each output $f(x)$ corresponds to exactly one input $x$.

If the second register is measured and collapses to $|f(x)\rangle$, the amplitude of observing $|z\rangle$ in the first register is:
$$
\text{Amplitude}(z) = \frac{(-1)^{x \cdot z}}{2^n}.
$$

The probability is:
$$
p(z) = \left| \frac{(-1)^{x \cdot z}}{2^n} \right|^2 = \frac{1}{2^{2n}}.
$$

Since there are $2^n$ possible values of $f(x)$, the total probability of measuring any particular $z$ is:
$$
\frac{1}{2^n}.
$$

**Conclusion:**  
The measurement outcomes $z$ are uniformly random.  
No structure is revealed.

---

### Case B: Two-to-One Function ($s \neq 0$)

Suppose measurement of the second register yields some value $|c\rangle$.
There are exactly two inputs $x_1$ and $x_2$ such that:
$$
f(x_1) = f(x_2) = c,
\quad x_2 = x_1 \oplus s.
$$

The first register collapses to:
$$
\frac{1}{\sqrt{2}} \left( |x_1\rangle + |x_2\rangle \right).
$$

After applying $H^{\otimes n}$, the amplitude of observing $|z\rangle$ is:
$$
\text{Amplitude}(z)
= \frac{1}{\sqrt{2^{n+1}}}
\left(
(-1)^{x_1 \cdot z} + (-1)^{x_2 \cdot z}
\right).
$$

Substituting $x_2 = x_1 \oplus s$:
$$
x_2 \cdot z = (x_1 \oplus s) \cdot z = x_1 \cdot z \oplus s \cdot z.
$$

For the amplitude to be nonzero, we require:
$$
(-1)^{x_1 \cdot z} = (-1)^{x_2 \cdot z}
\quad \Rightarrow \quad
s \cdot z = 0 \pmod 2.
$$

Thus, **only those bitstrings $z$ satisfying $s \cdot z = 0$ can be observed**.

For such $z$, the amplitude becomes:
$$
\text{Amplitude}(z)
= \frac{2(-1)^{x_1 \cdot z}}{\sqrt{2^{n+1}}},
$$
and the probability is:
$$
p(z) = \left| \frac{2}{\sqrt{2^{n+1}}} \right|^2
= \frac{2}{2^n}.
$$

---

## 4. Recovering the Secret String $s$

Each execution of the circuit produces a random bitstring $z$ such that:
$$
z \cdot s = 0 \pmod 2.
$$

By repeating the experiment, we collect a set:
$$
\{ z_1, z_2, \dots, z_{n-1} \},
$$
which yields a system of linear equations over $\mathbb{F}_2$:
$$
z_1 \cdot s = 0,
$$
$$
z_2 \cdot s = 0,
$$
$$
\vdots
$$
$$
z_{n-1} \cdot s = 0.
$$

Solving this system using **Gaussian elimination modulo 2** uniquely determines the secret string $s$ (up to the promised conditions).

---

## 5. Key Insight

Simon’s algorithm achieves an **exponential speedup** over classical algorithms by exploiting quantum superposition and interference to reveal hidden linear structure in the oracle function.


# Explaination of code below

This section explains **purely mathematically and logically** what the algorithm and the oracle are doing, **exactly in the way implemented by the code below**.  
No programming knowledge is assumed here—only basic ideas of binary strings, XOR, and linear algebra over bits.

---

## 1. The Problem Setting (Simon’s Promise)

We are given a function

$$
f : \{0,1\}^3 \rightarrow \{0,1\}^3
$$

with a **promise**:

> There exists a *nonzero* secret string $s = 110$ such that for **all** input strings $x$,

$$
f(x) = f(x \oplus s)
$$

Here:
- $x = (x_0, x_1, x_2)$ is a 3-bit input  
- $\oplus$ denotes bitwise XOR (addition modulo 2)

This promise implies:
- The function is **2-to-1**  
- Inputs come in **pairs**  
- Each pair differs **exactly by the string $110$**

---

## 2. What “Hiding the Secret” Means

The oracle **does not output the secret string**.  
Instead, it hides the secret in the **symmetry of the function**:

- The oracle is designed so that it **cannot distinguish** $x$ from $x \oplus 110$  
- Both inputs are mapped to the **same output value**

Thus, the secret string $s$ is encoded as an **equivalence relation**:

$$
x \sim x \oplus 110
$$

---

## 3. Pair Structure Implied by $s = 110$

For 3-bit inputs, XOR with $110$ produces the following pairs:

| Input $x$ | Paired input $x \oplus 110$ |
|------------|-------------------------------|
| 000 | 110 |
| 001 | 111 |
| 010 | 100 |
| 011 | 101 |

The oracle assigns:
- The **same function value** to both elements of each pair  
- **Different values** to different pairs

The exact output values do not matter — only the pairing matters.

---

## 4. Reversible Oracle Representation

Quantum mechanics requires all operations to be **reversible**.  
Therefore, instead of computing $f(x)$ directly, the oracle acts as:

$$
|x\rangle |y\rangle \;\longrightarrow\; |x\rangle |y \oplus f(x)\rangle
$$

- The input register $|x\rangle$ is **preserved**  
- The target register accumulates $f(x)$ via XOR  
- This guarantees unitarity and reversibility

---

## 5. How the Oracle Uses the Input Bits

Let the input bits be:

$$
x = (x_0, x_1, x_2)
$$

Recall:

$$
x \oplus 110 = (x_0 \oplus 1,\, x_1 \oplus 1,\, x_2)
$$

So:
- Bits $x_0$ and $x_1$ **flip**  
- Bit $x_2$ **does not flip**

### Consequence for Oracle Design

To satisfy the promise:
- The oracle **must treat $x_0$ and $x_1$ symmetrically**  
- The oracle **may depend on $x_2$**, since it is unchanged by XOR with $s$

This constraint completely determines the oracle’s logic.

---

## 6. Logical Structure of the Oracle Function

The oracle constructs $f(x)$ using these principles:

### (a) Dependence on $x_2$

Since $x_2$ is invariant under $x \oplus 110$, the function may safely depend on it.  
Thus:  
- Some output bits are functions of $x_2$

### (b) Symmetric Treatment of $x_0$ and $x_1$

The oracle uses Boolean conditions that give the **same result** for:
- $(x_0, x_1) = (0,0)$ and $(1,1)$  
- $(x_0, x_1) = (0,1)$ and $(1,0)$

This symmetry ensures:

$$
f(x_0, x_1, x_2) = f(x_0 \oplus 1, x_1 \oplus 1, x_2)
$$

which is exactly:

$$
f(x) = f(x \oplus 110)
$$

---

## 7. Resulting Mathematical Property of the Oracle

After the oracle is applied to a quantum superposition, the system contains states of the form:

$$
|x\rangle|f(x)\rangle + |x \oplus 110\rangle|f(x)\rangle
$$

Each pair shares:
- The same function value  
- A relative phase that enables interference

This structure is the **entire reason** Simon’s algorithm works.

---

## 8. Role of the Hadamard Transform (Conceptual)

After the oracle, a Hadamard transform is applied to the input register.  
Mathematically, this converts the pairing structure into a constraint:

$$
y \cdot s = 0 \pmod{2}
$$

where:
- $y$ is the measured output from the input register  
- $s = 110$  
- “$\cdot$” denotes bitwise dot product modulo 2

For $s = 110$, this condition becomes:

$$
y_0 \oplus y_1 = 0
$$

---

## 9. What the Measurements Mean

Each measurement produces a random vector $y$ satisfying:

$$
y \cdot 110 = 0
$$

These vectors **do not equal the secret string**.  
Instead, each one gives a **linear equation** about the secret.  
By collecting several such equations, the secret string $s = 110$ can be uniquely determined using classical linear algebra over $\mathbb{F}_2$.

---

## 10. Final Conceptual Summary

- The oracle **does not compute** the secret string  
- The oracle **hides** the secret as a symmetry  
- Quantum interference transforms that symmetry into linear constraints  
- Measurement reveals vectors orthogonal to the secret  
- The secret is recovered by solving those constraints

In short:

> **The oracle encodes the secret in which inputs it treats as identical, not in the values it outputs.**

This is the precise mathematical idea implemented by the code below.


# Cirq

In [8]:
# Listing 3-11: Simon’s Algorithm using Cirq

import cirq
import numpy as np


def oracle(input_qubits, target_qubits, circuit):
    """
    Oracle implementing a hidden secret string s = 110.
    The oracle satisfies: f(x) = f(x ⊕ s)
    """

    # Encode dependence on input_qubits[2]
    ''' 
    This part encodes the dependence of the oracle on the input bit x2.
    x2 is the bit that remains unchanged when XORed with the secret string s = 110.
    The target qubits are updated using CNOT and X gates so that the function output depends on x2,
    while ensuring that inputs differing by the secret s produce the same output.
    '''
    circuit.append(cirq.CNOT(input_qubits[2], target_qubits[1]))
    circuit.append(cirq.X(target_qubits[0]))
    circuit.append(cirq.CNOT(input_qubits[2], target_qubits[0]))

    # Multi-controlled operations to complete the oracle logic
    ''' 
    These are Toffoli (CCNOT) and X gates that handle the input bits x0 and x1.
    The gates are applied in a symmetric fashion so that the oracle cannot distinguish
    between inputs x and x ⊕ s. This ensures that f(x) = f(x ⊕ s) for all x.
    The intermediate X gates flip input bits to create the necessary control conditions for the multi-controlled NOTs.
    '''
    circuit.append(cirq.CCNOT(input_qubits[0], input_qubits[1], target_qubits[0]))

    circuit.append(cirq.X(input_qubits[0]))
    circuit.append(cirq.X(input_qubits[1]))

    circuit.append(cirq.CCNOT(input_qubits[0], input_qubits[1], target_qubits[0]))

    circuit.append(cirq.X(input_qubits[0]))
    circuit.append(cirq.X(input_qubits[1]))

    circuit.append(cirq.X(target_qubits[0]))
    
    '''
    After all the operations, the oracle has updated the target qubits to encode f(x)
    while preserving reversibility and ensuring the Simon promise holds.
    '''
    return circuit


def simons_algorithm_circuit(num_qubits=3, copies=1000):
    """
    Build and simulate Simon's Algorithm circuit.

    Parameters
    ----------
    num_qubits : int
        Number of input qubits (n)
    copies : int
        Number of repetitions for measurement statistics
    """

    # Define input and target registers
    ''' 
    Input qubits hold the input string x.
    Target qubits hold the function value f(x) as defined by the oracle.
    '''
    input_qubits = [cirq.LineQubit(i) for i in range(num_qubits)]
    target_qubits = [cirq.LineQubit(i) for i in range(num_qubits, 2 * num_qubits)]

    circuit = cirq.Circuit()

    # Step 1: Create equal superposition over input register
    ''' 
    Apply Hadamard gates to each input qubit to create an equal superposition of all possible input states |x⟩.
    This prepares the input for quantum parallel evaluation of f(x).
    '''
    circuit.append(cirq.H.on_each(*input_qubits))

    # Step 2: Apply Simon oracle
    ''' 
    Apply the oracle to the superposition state.
    The oracle encodes f(x) in the target register while creating entanglement
    between inputs x and their corresponding function values f(x).
    '''
    circuit = oracle(input_qubits, target_qubits, circuit)

    # Step 3: Apply Hadamard again on input register
    ''' 
    Apply Hadamard gates again to the input qubits.
    This transforms the entangled superposition into a state where measuring the input qubits
    gives vectors y orthogonal to the secret string s (y · s = 0 mod 2).
    '''
    circuit.append(cirq.H.on_each(*input_qubits))

    # Step 4: Measure both registers
    ''' 
    Measure all qubits (input + target).
    Only the input qubits are required to extract s, but measuring both is harmless.
    The measurements collapse the quantum state to classical bitstrings,
    giving outcomes consistent with y · s = 0.
    '''
    circuit.append(
        cirq.measure(*(input_qubits + target_qubits), key='Z')
    )

    print("Circuit Diagram for Simon's Algorithm:\n")
    print(circuit)

    # Simulate the circuit
    ''' 
    Use Cirq's Simulator to run the quantum circuit multiple times (copies).
    Each repetition produces one measurement outcome.
    '''
    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=copies)

    # Histogram of measurement outcomes
    ''' 
    Count how many times each classical bitstring appears.
    This gives the probability distribution of the measured states.
    '''
    raw_histogram = dict(result.histogram(key='Z'))

    # Convert results to fixed-length binary strings
    ''' 
    Convert each integer key from the histogram to a binary string representing the measured bits.
    This makes it easier to interpret and compare the outcomes.
    '''
    formatted_results = {}
    total_bits = 2 * num_qubits

    for key, count in raw_histogram.items():
        binary_key = format(key, f'0{total_bits}b')
        formatted_results[binary_key] = count

    print("\nMeasurement Results (binary → counts):\n")
    for k, v in formatted_results.items():
        print(f"{k} : {v}")


if __name__ == '__main__':
    simons_algorithm_circuit()


Circuit Diagram for Simon's Algorithm:

                  ┌──┐
0: ───H────────────@─────X───@───X───H───M('Z')───
                   │         │           │
1: ───H────────────@─────X───@───X───H───M────────
                   │         │           │
2: ───H───@───@────┼H────────┼───────────M────────
          │   │    │         │           │
3: ───X───┼───X────X─────────X───X───────M────────
          │                              │
4: ───────X──────────────────────────────M────────
                                         │
5: ──────────────────────────────────────M────────
                  └──┘

Measurement Results (binary → counts):

111010 : 74
110110 : 66
000110 : 54
000000 : 80
000010 : 54
111000 : 79
001000 : 55
111100 : 50
110100 : 62
110010 : 65
111110 : 52
000100 : 61
001100 : 63
001010 : 69
110000 : 59
001110 : 57


## Extracting the Secret String from the Measurement Outcomes

From the measurement results of Simon’s algorithm, the secret string $s$ can be recovered directly by analyzing pairs of outcomes that correspond to the **same oracle output**.

Each measurement outcome represents a combined state of the form:
$$
|x\rangle |f(x)\rangle,
$$
where the **first 3 bits** correspond to the input string $x$, and the **last 3 bits** correspond to the function value $f(x)$.

Consider the two observed outcomes:
$$
111010 \quad \text{and} \quad 001010.
$$

Splitting each outcome into input and output parts:
- For $111010$:  
  $x_1 = 111$, $f(x_1) = 010$
- For $001010$:  
  $x_2 = 001$, $f(x_2) = 010$

Since both outcomes yield the **same output value** $f(x_1) = f(x_2) = 010$, the oracle promise guarantees that these inputs satisfy:
$$
x_2 = x_1 \oplus s.
$$

Therefore, the secret string $s$ can be obtained by taking the bitwise XOR (modulo-2 addition) of the two inputs:
$$
s = x_1 \oplus x_2 = 111 \oplus 001.
$$

Evaluating the XOR:
$$
111 \oplus 001 = 110.
$$

Hence, the recovered secret string is:
$$
s = 110,
$$
which exactly matches the secret code embedded in the oracle.

This illustrates how Simon’s algorithm reveals the hidden period by exploiting quantum interference and correlations between input states that produce identical function outputs.


In [11]:
# Simon’s Algorithm using Qiskit (AerSimulator)

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

def oracle(input_qubits, target_qubits, circuit):
    """
    Oracle implementing a hidden secret string s = 110.
    The oracle satisfies: f(x) = f(x ⊕ s)
    """

    # Encode dependence on input_qubits[2]
    ''' 
    Dependence on x2: flip target qubits according to x2 while keeping the Simon promise
    '''
    circuit.cx(input_qubits[2], target_qubits[1])
    circuit.x(target_qubits[0])
    circuit.cx(input_qubits[2], target_qubits[0])

    # Multi-controlled operations to complete the oracle logic
    ''' 
    Symmetric handling of x0 and x1 using Toffoli (CCX) and X gates
    to ensure f(x) = f(x ⊕ s) for all x
    '''
    circuit.ccx(input_qubits[0], input_qubits[1], target_qubits[0])

    circuit.x(input_qubits[0])
    circuit.x(input_qubits[1])

    circuit.ccx(input_qubits[0], input_qubits[1], target_qubits[0])

    circuit.x(input_qubits[0])
    circuit.x(input_qubits[1])

    circuit.x(target_qubits[0])

    return circuit

def simons_algorithm_circuit(num_qubits=3, shots=1000):
    """
    Build and simulate Simon's Algorithm circuit in Qiskit.

    Parameters
    ----------
    num_qubits : int
        Number of input qubits (n)
    shots : int
        Number of repetitions for measurement statistics
    """

    # Define input and target registers
    ''' 
    Input qubits hold x, target qubits hold f(x)
    '''
    circuit = QuantumCircuit(2*num_qubits, 2*num_qubits)

    input_qubits = list(range(num_qubits))
    target_qubits = list(range(num_qubits, 2*num_qubits))

    # Step 1: Create equal superposition over input register
    ''' 
    Hadamard gates create superposition |x⟩ over all input states
    '''
    for q in input_qubits:
        circuit.h(q)

    # Step 2: Apply Simon oracle
    circuit = oracle(input_qubits, target_qubits, circuit)

    # Step 3: Apply Hadamard again on input register
    for q in input_qubits:
        circuit.h(q)

    # Step 4: Measure all qubits
    for i in range(2*num_qubits):
        circuit.measure(i, i)

    print("Circuit Diagram for Simon's Algorithm:\n")
    print(circuit.draw(output='text'))

    # Simulate the circuit
    simulator = AerSimulator()
    result = simulator.run(circuit, shots=shots).result()
    counts = result.get_counts()

    # Format counts to fixed-length binary strings
    formatted_results = {}
    for key, value in counts.items():
        # Qiskit returns key in reversed order by default
        key = key.replace(" ", "")[::-1]  # remove spaces and reverse
        key = key.zfill(2*num_qubits)
        formatted_results[key] = value

    print("\nMeasurement Results (binary → counts):\n")
    for k, v in formatted_results.items():
        print(f"{k} : {v}")

    return formatted_results

if __name__ == '__main__':
    simons_algorithm_circuit()


Circuit Diagram for Simon's Algorithm:

     ┌───┐               ┌───┐        ┌───┐┌───┐┌─┐   
q_0: ┤ H ├────────────■──┤ X ├─────■──┤ X ├┤ H ├┤M├───
     ├───┤            │  ├───┤     │  ├───┤├───┤└╥┘┌─┐
q_1: ┤ H ├────────────■──┤ X ├─────■──┤ X ├┤ H ├─╫─┤M├
     ├───┤            │  ├───┤┌─┐  │  └───┘└───┘ ║ └╥┘
q_2: ┤ H ├──■────■────┼──┤ H ├┤M├──┼─────────────╫──╫─
     ├───┤  │  ┌─┴─┐┌─┴─┐└───┘└╥┘┌─┴─┐┌───┐ ┌─┐  ║  ║ 
q_3: ┤ X ├──┼──┤ X ├┤ X ├──────╫─┤ X ├┤ X ├─┤M├──╫──╫─
     └───┘┌─┴─┐└┬─┬┘└───┘      ║ └───┘└───┘ └╥┘  ║  ║ 
q_4: ─────┤ X ├─┤M├────────────╫─────────────╫───╫──╫─
      ┌─┐ └───┘ └╥┘            ║             ║   ║  ║ 
q_5: ─┤M├────────╫─────────────╫─────────────╫───╫──╫─
      └╥┘        ║             ║             ║   ║  ║ 
c: 6/══╩═════════╩═════════════╩═════════════╩═══╩══╩═
       5         4             2             3   0  1 

Measurement Results (binary → counts):

111100 : 63
111010 : 57
110000 : 73
000000 : 64
110010 : 57
111000 : 71
111110 : 63
001010 : 5